# TODO：概要

# ライブラリのインポート

In [31]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
)

# 環境変数の取得

In [32]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")

# クライアントの初期化

In [33]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# ユーティリティ関数

In [34]:
def agent_run_outputs(thread_id, project_client, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    """
    messages = project_client.agents.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    for message in messages:
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{message.role.upper()}: {txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                project_client.agents.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))

# ツールの定義

In [35]:
mcp_tool = McpTool(
    server_label="MicrosoftDocs",
    server_url="https://learn.microsoft.com/api/mcp",
)


# エージェントの作成

In [36]:
mcp_agent = project_client.agents.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="mcp_agent",
    instructions=(
        "あなたは計算や図表の出力に特化したアシスタントです。"
        "利用可能なツールを確認し、必要に応じて使用してください。"    
        "なお、グラフを描画する際は日本語が文字化けするため、ラベルやタイトルは英語で出力してください。"    
    ),
    description=(
        "Code Interpreter を利用して、計算や図表の出力に特化した分析アシスタントです。"
    ),
    tools=mcp_tool.definitions,
)
print(f"Created Agent. AGENT_ID: {mcp_agent.id}")


Created Agent. AGENT_ID: asst_R5mtVlX34QX4R86cxYDXWHG3


# スレッドの作成

In [37]:
# Thread の作成
thread = project_client.agents.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_DE3MEysHSTQ6MCs1VnLnvZJx


# ユーザーメッセージの追加

In [38]:
# メッセージの追加
user_message = "Azure AI Foundry Agent Service の最新情報を教えてください。"

message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_s1jgeud6iPsKi1e1UofMLaPX


# Run の実行

In [39]:
run = project_client.agents.runs.create(
    thread_id=thread.id,
    agent_id=mcp_agent.id
)

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        tool_calls = run.required_action.submit_tool_approval.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            project_client.agents.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_approvals = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredMcpToolCall):
                try:
                    print(f"Approving tool call: {tool_call}")
                    tool_approvals.append(
                        ToolApproval(
                            tool_call_id=tool_call.id,
                            approve=True,
                            headers=mcp_tool.headers,
                        )
                    )
                except Exception as e:
                    print(f"Error approving tool_call {tool_call.id}: {e}")

        print(f"tool_approvals: {tool_approvals}")
        if tool_approvals:
            project_client.agents.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_approvals=tool_approvals
            )

    print(f"Current run status: {run.status}")

Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Approving tool call: {'id': 'call_0GfHumyk8RbBOWNtpiAccmqF', 'type': 'mcp', 'arguments': '{"question":"Azure AI Foundry Agent Service æ\x9c\x80æ\x96°æ\x83\x85å\xa0±"}', 'name': 'microsoft_docs_search', 'server_label': 'MicrosoftDocs'}
tool_approvals: [{'tool_call_id': 'call_0GfHumyk8RbBOWNtpiAccmqF', 'approve': True, 'headers': {}}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED


In [40]:
agent_run_outputs(thread.id, project_client)

USER: Azure AI Foundry Agent Service の最新情報を教えてください。
ASSISTANT: Azure AI Foundry Agent Serviceの最新情報（2025年6月時点）は以下の通りです。

### 主な新機能とアップデート

#### 2025年6月
- **Deep Research tool**  
  - Azure OpenAIの`o3-deep-research`モデルを使い、Bing検索によるグラウンディングを一括したマルチステップリサーチを実現する新ツールです。  
  - [Deep Research tool詳細 (英語)](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/deep-research)
- **Model Context Protocol (MCP) tool**  
  - 外部MCPサーバーのツールと接続することで、エージェントの機能拡張が可能になりました。  
  - [MCP tool詳細 (英語)](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol)

#### 2025年5月
- **一般提供(GA)開始**
  - Azure AI Foundry Agent Serviceが正式提供となりました。
  - Visual Studio Code拡張機能で、ネイティブにエージェントのデプロイや設定が可能に。
  - **Connected agents**でマルチエージェント・システムを容易に構築。
  - **Trace agents**により、各実行スレッドの入出力の詳細トレースが可能。
  - **Logic Apps連携**でイベントトリガーによるエージェント自動実行対応。
  - 新規ツール追加（Bing Custom Search, Morningstarなど）。

#### 2025年4月
- **Azure Monitor統合**  
  - エージェントのファイルインデックス数や実行回数などの指標をモニタリング可能に。
- *

※ ここで、Azure AI Foundry 上の Web UI からエージェントのトレースを確認してみよう。

# Agent ID を .env ファイルに保存
※ 今回作成したエージェントを、後続の Connected Agents のハンズオン演習で使用するため永続化します。

In [41]:
# 変数の定義
agent_env_key = "FOUNDRY_MCP_AGENT_ID"
agent_env_value = mcp_agent.id

# .envファイルのパスを自動探索
env_path = find_dotenv()  # 見つからなければ''を返す
if not env_path:
    raise FileNotFoundError(".envファイルが見つかりませんでした。")

# AGENT_ID を .env ファイルに追記
with open(env_path, "a", encoding="utf-8") as f:
    f.write(f'\n{agent_env_key}="{agent_env_value}"')

print(f'.envファイルに {agent_env_key}=\"{agent_env_value}\" を追記しました。')

# .envの自動読み込み（以降os.environで参照可能）
load_dotenv(env_path)


.envファイルに FOUNDRY_MCP_AGENT_ID="asst_R5mtVlX34QX4R86cxYDXWHG3" を追記しました。


True